##### pysam docs

https://pysam.readthedocs.io/en/latest/index.html

In [269]:
import numpy as np
import pysam

### Grabbing the SAM file and grabbing all the seqs

In [270]:
samfile_A = pysam.AlignmentFile("pol_a.sam", "r")
all_reads_A = samfile_A.fetch()

In [271]:
samfile_B = pysam.AlignmentFile("pol_b.sam", "r")
all_reads_B = samfile_B.fetch()

### Gathering reads 

In [272]:
read_A = next(all_reads_A)
read_B = next(all_reads_B)

In [273]:
#dir(read_A)

In [274]:
print(read_A.reference_name)
print(read_A.qname)
print(read_A.query_alignment_sequence)
print(read_A.query_alignment_length)
print(read_A.qual)
print(read_A.mapq)
print(read_A.qstart)
print(read_A.qend)

B.CH.2002.HIV_CH_BID-V3527_2002.JQ403021
B.CH.2002.HIV_CH_BID-V3527_2002.JQ403021-10000
GTAATACAAGATAATAGTGACATAAAAGTAGTGCCAAGAAGAAAAGCAAAGATCATTAGGGATTATGGAAAACAGATGGCAGGTGATGATTGTGTGGCAGGTAGACAGGATGAGGATTAG
120
GGGGCGGGGGGGGGGGGGGGGG>GGGGGGGGGGGGDGGGGGGEEGGGGGGDGGGG=GGGGGGGGCGGGFGFGGEGGGG1GGGGGGGGGGGGCGGGGCDGGGGGGGGGG//GGFGFCCCCC
99
0
120


### These will be the steps in the selection process for selecting a seq to recombine with

#### The array length will be based on the frequencies of the seqs from a ref in your larger sample

In [275]:
master = {.55: ("pol_a",samfile_A), .45: ("pol_b",samfile_B)}
f = list(master.keys())
freqs = np.array(f)

#### chose a reference to sample from given the distribution above, the answer (i) will be 0 indexed

In [276]:
np.random.seed(1)
i = np.random.choice(2, 1, True, freqs)[0]
original_ref = f[i]
ref_1_name, ref_1_file = master[original_ref][0], master[original_ref][1]
print("We will grab a read from ", ref_1_name, " to Artificially Recombine")

We will grab a read from  pol_a  to Artificially Recombine


##### This output says "select the ref at position 0"

#### Now create a new frequency weighting, EXCLUDING the previously selected choice, this will tell you which other reference you will recombine with, this is also 0 indexed

In [277]:
freq_reweight_list = freqs[np.arange(len(freqs)) !=i]

freq_reweight = freq_reweight_list / np.sum(freq_reweight_list)
j = np.random.choice(len(freq_reweight), 1, True, freq_reweight)[0]

ref_to_select = freq_reweight_list[j]
#print(ref_to_select)
ref_2_name, ref_2_file = master[ref_to_select][0], master[ref_to_select][1]

print("We will now draw a read from: ", ref_name, " to Artificially Recombine with ", ref_1_name)

We will now draw a read from:  pol_b  to Artificially Recombine with  pol_a


##### This output says "create a new list that does not include the previous ref, now which is most likely to be selected, and grab that. In this case it is the ref at position 0 (which must be the other reference because there are only two and starting with one then excluding it only leaves you with one left)"

### Introduce the other SAM reads here to start pulling from

In [278]:
good_reads = list(filter(
    lambda read: read.reference_start == read_A.reference_start and read.reference_end == read_A.reference_end,
    ref_2_file
))

In [308]:
seq_read_2 = good_reads[0].seq
seq_read_1 = read_A.seq
print("1 --> ",seq_read_1)
print("2 --> ",seq_red_2)

1 -->  GTAATACAAGATAATAGTGACATAAAAGTAGTGCCAAGAAGAAAAGCAAAGATCATTAGGGATTATGGAAAACAGATGGCAGGTGATGATTGTGTGGCAGGTAGACAGGATGAGGATTAG
2 -->  GTAATTCAAGATAATAGTGACATAAAAGTAGTGCCAAGAAGAAAATCAAAGATCATTAGGGATTATGGAAAACAGATGGCAGGTGATGATTGTGTGGCAGGTAGACAGGATGAGGATTAA


In [309]:
len(seq_red_2) == len(seq_read_1)

True

In [310]:
grabber = np.random.randint(len(seq_red_2))

In [311]:
grabber

79

In [323]:
indexed_seq_1_start = seq_read_1[:grabber]
indexed_seq_2_end = seq_read_2[grabber:]

In [324]:
print("total len (should be len of read) -->", len(indexed_seq_2_end) + len(indexed_seq_1_start))

print("len of 1--> ",len(indexed_seq_1_start))
print("len of 2 -->",len(indexed_seq_2_end))

total len (should be len of read) --> 120
len of 1-->  79
len of 2 --> 41


120

In [326]:
indexed_seq_2_end

'CAGGTGATGATTGTGTGGCAGGTAGACAGGATGAGGATTAA'